# Obtain the block_ranking

In [1]:
import pandas as pd
import pickle
import networkx as nx
import queue
import itertools
import numpy as np
import time

Load the graph by the pickle file created through the RQ1:

In [2]:
with open("Graph", "rb") as f:
    graph = pickle.load(f)

In [3]:
df = pd.read_csv('wiki-topcats-page-names.txt',sep='\n', header=-1, names=["Name"])

#remove the index from the name:
df["Name"]=df.apply(lambda x: " ".join(x["Name"].split()[1:]),axis=1)
df.head()

,Name
0,Chiasmal syndrome
1,Kleroterion
2,Pinakion
3,LyndonHochschildSerre spectral sequence
4,Zariski's main theorem


In [4]:
n=len(df)
n_graph = graph.number_of_nodes()
print("Total number of nodes:",n)
print("Number of nodes by the graph:",n_graph)

Total number of nodes: 1791489
Number of nodes by the graph: 461193


We oserve that the great majority of the nodes are not in the graph (isolated points)

The structure of the file containing the categories is:

    Category:NameOfTheCategory; indexes of the nodes belonging to the category separeted by space

In [5]:
category = pd.read_csv("wiki-topcats-categories.txt", header = None, sep = "/n", names=["all"],engine="python")

#Reaching the name of the category:
#Splitting by ";" and the taking the first part, splitting by ":" and taking the second part
category["Name"]=category.apply(lambda x: x["all"].split(";")[0].split(":")[1], axis=1)

#Reaching the list pf nodes:
#Splitting by ":" and taking the second part and split by " "
category["nodes"]= category.apply(lambda x: list(map(int,x["all"].split(";")[1].split())),axis=1)

#Just change the order of the columns and drop the columns "all"
category= category[["Name","nodes"]]

category.head()

,Name,nodes
0,Buprestoidea,"[301, 302, 303, 304, 305, 306, 307, 308, 309, ..."
1,People_from_Worcester,"[1056, 1057, 1058, 1059, 1060, 60971, 76515, 7..."
2,Skin_conditions_resulting_from_physical_factors,"[971, 973, 1166, 1167, 1168, 1169, 1170, 1171,..."
3,Visual_kei_bands,"[1297, 1300, 1311, 1312, 1313, 1314, 1315, 131..."
4,Japanese_rock_music_groups,"[1297, 1300, 1313, 1314, 1315, 1316, 1319, 132..."


Now we can keep (as required) just the category with more than 3500 nodes:

In [6]:
cat = category.loc[category.apply(lambda x: len(x["nodes"]), axis = 1) > 3500]

In [7]:
c = len(cat)
c

35

In [8]:
cat.head()

,Name,nodes
868,English_footballers,"[22860, 28411, 28961, 28979, 29264, 29573, 295..."
869,The_Football_League_players,"[14003, 23536, 27109, 27348, 27459, 27989, 280..."
876,Association_football_forwards,"[26876, 26877, 26879, 26887, 26892, 26904, 269..."
898,Association_football_goalkeepers,"[26900, 26909, 26917, 26960, 26966, 26984, 270..."
900,Association_football_midfielders,"[14003, 15291, 23536, 26880, 26882, 26885, 268..."


In [9]:
lengths=[]
for i,val in cat.iterrows():
    lengths.append(len(val["nodes"]))

In order to reduce the running time, let's take the smallest category as the input category:

In [10]:
idx_min =np.argmin(np.array(lengths))
C0 = cat.iloc[idx_min]["Name"]
print(C0)  #Name of the input category

English_television_actors


In [11]:
nodes_C0 = list(cat.loc[cat["Name"]==C0]["nodes"])[0]
print(len(nodes_C0))  #Number of its nodes

3501


Write the function that computes the BFS. That algorithm computes the shortest path between one source node and all the other nodes of the graph for an unweighted graph

In [12]:
def breadth_first_search(G,a):
    visited=[False]*n
    visited[a]=True
    q = queue.Queue()
    q.put(a)
    d={}
    d[a]=0
    while not q.empty():
        v=q.get()
        for i in list(G.neighbors(v)):
            if not visited[i]:
                q.put(i)
                visited[i]=True
                d[i] = d[v]+1
    return d

In [13]:
a = time.time()
bfs1 =nx.single_source_shortest_path_length(G=graph,source=52)
b = time.time()
our_bfs =breadth_first_search(graph, 52)
c = time.time()
print(bfs1 == our_bfs)
print("The running time of the algorithm from the library is",b-a)
print("The running time of our algorithm",c-b)

True
The running time of the algorithm from the library is 1.4409425258636475
The running time of our algorithm 1.6801321506500244


Our BFS is identical to the algorithm from the library (so it is correct). Also we see that the running times are very similar.

Now we need to compute the dictionary of the distances (using BFS) with source every node belonging to the category $C_0$. 

We would create a dictionary of this format:

- The key are all the nodes of the graph
- For each key the value is a list, where each element is the lenght of the shortest path with one of the nodes of the category $C_0$

But we would encounter a memory error (the previous dictionary has about 400k keys, each one with value a list of about 3.5k elements).

Then we store as values (instead of a list) a nested dictionary, with keys the distances and values the FREQUENCIES with which each distance appear.

In [16]:
# !!! This cell would take 2 hours, DO NOT RUN IT !!!
# Instead if you want you can upload the "final_dict" (2 cells later)

final_dict=dict((i,{}) for i in graph.nodes)
for i in nodes_C0:
    if i in graph:
        d = breadth_first_search(graph,i)
        for j in d:
            final_dict[j][d[j]] = final_dict[j].get(d[j],0) +1

In [17]:
with open("final_dict","wb") as f:
    pickle.dump(final_dict,f)

Now we can load it:

In [14]:
with open("final_dict","rb") as f:
    final_dict = pickle.load(f)

In [15]:
final_dict

{52: {},
 62: {},
 64: {},
 66: {},
 74: {},
 95: {5: 1835, 6: 1080, 4: 266, 7: 59, 3: 4, 8: 2, 9: 1},
 103: {},
 104: {6: 1771, 5: 1236, 7: 160, 8: 13, 4: 64, 3: 2, 9: 1},
 105: {},
 107: {5: 2041, 4: 575, 6: 591, 7: 28, 3: 10, 8: 2},
 108: {},
 112: {},
 113: {},
 122: {},
 126: {},
 133: {},
 134: {4: 1820, 5: 1082, 6: 95, 3: 236, 7: 6, 2: 7, 8: 1},
 137: {6: 1407, 4: 109, 5: 1548, 7: 171, 8: 10, 3: 1, 9: 1},
 153: {5: 2009, 6: 633, 4: 524, 7: 35, 3: 44, 8: 2},
 154: {5: 1886, 4: 651, 6: 636, 7: 26, 3: 45, 8: 2, 2: 1},
 155: {},
 156: {},
 158: {5: 1997, 4: 480, 7: 33, 6: 723, 8: 2, 3: 12},
 160: {},
 163: {},
 166: {6: 1629, 7: 1378, 8: 131, 5: 100, 9: 8, 4: 1},
 167: {6: 1502, 7: 1379, 8: 134, 9: 5, 5: 215, 4: 10, 3: 1, 10: 1},
 173: {4: 1950, 5: 882, 3: 322, 2: 16, 6: 69, 7: 7, 1: 1},
 174: {4: 1983, 5: 937, 3: 250, 6: 67, 7: 6, 2: 3, 8: 1},
 185: {5: 1429, 6: 1569, 7: 164, 8: 9, 4: 70, 3: 5, 9: 1},
 186: {5: 1429, 6: 1569, 7: 164, 8: 9, 4: 70, 3: 5, 9: 1},
 190: {},
 195: {5: 19

Let's see how it looks like, just for two nodes:

In [17]:
{95: final_dict[95], 103:final_dict[103], 104:final_dict[104]}

{95: {5: 1835, 6: 1080, 4: 266, 7: 59, 3: 4, 8: 2, 9: 1},
 103: {},
 104: {6: 1771, 5: 1236, 7: 160, 8: 13, 4: 64, 3: 2, 9: 1}}

Now for each category $C_i$, we can easily obtain the __ShortestPath($C_0$,$C_i$)__ just taking all the dictionaries for every node belonging to $C_i$, and merging them.

Once obtained the set  ShortestPath($C_0$,$C_i$), we just need to take the median of it.

We could compute the median using `statistics.median`, but in the code of that function it is used sorted(list) instead of list.sort().

Now the problem is that for the category Living_people which has 400k nodes, the list shortest_path is very large, and if we use sorted() we are creating a copy of this, that could create a __Memory error__. So we have to create a little function to compute the median by ourself:

In [18]:
def median(l):
    l.sort()
    middle = len(l)//2
    return l[middle]

In [19]:
medians={}
list_of_names = cat.loc[cat["Name"]!=C0]["Name"]
for Ci in list_of_names:
    nodes_Ci = list(cat.loc[cat["Name"]==Ci]["nodes"])[0]
    shortest_path=[]
    for node in nodes_Ci:
        try:
            d = final_dict[node]
            for lenght in d:
                shortest_path.extend(d[lenght]*[lenght])
        except:
            continue
    medians[Ci]= median(shortest_path)

In [23]:
medians

{'English_footballers': 6,
 'The_Football_League_players': 6,
 'Association_football_forwards': 6,
 'Association_football_goalkeepers': 6,
 'Association_football_midfielders': 6,
 'Association_football_defenders': 6,
 'Living_people': 6,
 'Year_of_birth_unknown': 6,
 'Harvard_University_alumni': 6,
 'Major_League_Baseball_pitchers': 6,
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 6,
 'Indian_films': 6,
 'Year_of_death_missing': 6,
 'English_cricketers': 6,
 'Year_of_birth_missing_(living_people)': 6,
 'Rivers_of_Romania': 7,
 'Main_Belt_asteroids': 8,
 'Asteroids_named_for_people': 7,
 'English-language_albums': 5,
 'British_films': 5,
 'English-language_films': 5,
 'American_films': 5,
 'Fellows_of_the_Royal_Society': 6,
 'People_from_New_York_City': 5,
 'American_Jews': 5,
 'American_television_actors': 5,
 'American_film_actors': 5,
 'Debut_albums': 6,
 'Black-and-white_films': 5,
 'Year_of_birth_missing': 6,
 'Place_of_birth_missing_(living_people)': 6,
 

In [21]:
with open("medians","wb") as f:
    pickle.dump(medians,f)

In [22]:
with open("medians","rb") as f:
    medians = pickle.load(f)

Just sort the dictionary by values:

In [24]:
block_ranking = [k for k, v in sorted(medians.items(), key=lambda x: x[1])]

In [25]:
block_ranking

['English-language_albums',
 'British_films',
 'English-language_films',
 'American_films',
 'People_from_New_York_City',
 'American_Jews',
 'American_television_actors',
 'American_film_actors',
 'Black-and-white_films',
 'Article_Feedback_Pilot',
 'American_military_personnel_of_World_War_II',
 'English_footballers',
 'The_Football_League_players',
 'Association_football_forwards',
 'Association_football_goalkeepers',
 'Association_football_midfielders',
 'Association_football_defenders',
 'Living_people',
 'Year_of_birth_unknown',
 'Harvard_University_alumni',
 'Major_League_Baseball_pitchers',
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
 'Indian_films',
 'Year_of_death_missing',
 'English_cricketers',
 'Year_of_birth_missing_(living_people)',
 'Fellows_of_the_Royal_Society',
 'Debut_albums',
 'Year_of_birth_missing',
 'Place_of_birth_missing_(living_people)',
 'Windows_games',
 'Rivers_of_Romania',
 'Asteroids_named_for_people',
 'Main_Belt_asteroids']

# Sort the nodes in each category